In [2]:
#Import dependencies
import pandas as pd
import requests

In [19]:
#Retrieve the data
crypto_data = pd.read_csv("Resources/crypto_data.csv")
crypto_data = crypto_data.drop("CoinName", axis=1)
crypto_data

,Unnamed: 0,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,X11,True,PoW/PoS,NaN,2300000000
2,404,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1247,XBC,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,SHA-256,False,PoW/PoS,NaN,21000000


In [20]:
#Discard all cryptocurrencies that are not being traded
active_traders = crypto_data[crypto_data.IsTrading==True]
active_traders

,Unnamed: 0,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,X11,True,PoW/PoS,NaN,2300000000
2,404,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,SERO,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,BDX,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,True,PoW,7.296538e+06,21000000


In [21]:
#Drop the target feature
drop_feature = active_traders.drop("IsTrading", axis=1)
drop_feature

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
1,365,X11,PoW/PoS,NaN,2300000000
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
1243,SERO,Ethash,PoW,NaN,1000000000
1244,UOS,SHA-256,DPoI,NaN,1000000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000


In [22]:
#Remove all rows that have at least one null value
no_nulls = drop_feature.dropna()
no_nulls

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000


In [23]:
#Filter for cryptocurrencies that have been mined.
mined_coins = no_nulls[no_nulls.TotalCoinsMined>0]
mined_coins

,Unnamed: 0,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,Scrypt,PoW/PoS,4.199995e+01,42
2,404,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Equihash,PoW,7.296538e+06,21000000


In [25]:
#Convert categorical values to numerical values
categorical_features = mined_coins.dtypes[mined_coins.dtypes==object]
categorical_features

Unnamed: 0         object
Algorithm          object
ProofType          object
TotalCoinSupply    object
dtype: object

In [28]:
encoded_features = pd.get_dummies(data=mined_coins, columns=["Algorithm", "ProofType"])
encoded_features.columns

Index(['Unnamed: 0', 'TotalCoinsMined', 'TotalCoinSupply',
       'Algorithm_1GB AES Pattern Search', 'Algorithm_536',
       'Algorithm_Argon2d', 'Algorithm_BLAKE256', 'Algorithm_Blake',
       'Algorithm_Blake2S', 'Algorithm_Blake2b', 'Algorithm_C11',
       'Algorithm_Cloverhash', 'Algorithm_Counterparty',
       'Algorithm_CryptoNight', 'Algorithm_CryptoNight Heavy',
       'Algorithm_CryptoNight-V7', 'Algorithm_Cryptonight-GPU',
       'Algorithm_DPoS', 'Algorithm_Dagger', 'Algorithm_Dagger-Hashimoto',
       'Algorithm_ECC 256K1', 'Algorithm_Equihash',
       'Algorithm_Equihash+Scrypt', 'Algorithm_Ethash', 'Algorithm_Exosis',
       'Algorithm_Green Protocol', 'Algorithm_Groestl', 'Algorithm_HMQ1725',
       'Algorithm_HybridScryptHash256', 'Algorithm_IMesh',
       'Algorithm_Jump Consistent Hash', 'Algorithm_Keccak',
       'Algorithm_Leased POS', 'Algorithm_Lyra2RE', 'Algorithm_Lyra2REv2',
       'Algorithm_Lyra2Z', 'Algorithm_M7 POW', 'Algorithm_Multiple',
       'Algorithm_

In [ ]:
#Standardize the dataset 